**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 2 - Project Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a project account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the project account;

6 - Queries the blockchain again to confirm the succesfull details setting.

Every account succesfully created is dumped to `datasets/projects.json` for later use and evidence collection.  

In [1]:
# from name_generator import left, right
from utilities import *
from project_id_generator import * # - Random generated synthetic project ID
from iroha_helper import *
from loguru import logger
from dump_to_json import ProjectAccount, update_or_append_project_metadata
from ipfs_functions import *
from synthetic_project_json_ld import *
from new_helper import extract_user_json_ld_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [3]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.  

In [ ]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2 - Creates a project account into the blockchain;

In [8]:
project_private_key = IrohaCrypto.private_key()
project_public_key = IrohaCrypto.derive_public_key(project_private_key).decode("utf-8")

project_id = generate_project_id()

logger.info(f"Creating project with name: {project_id}")

# project_account = ProjectAccount(
#     project_id=f"{project_id}@{DOMAIN}",
#     public_key=project_public_key
# )

logger.info(f"Creating project with name: {project_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_project_account(create_account_contract_address, project_id, DOMAIN, project_public_key)


2025-01-25 12:10:23.246 | INFO     | __main__:<module>:6 - Creating project with name: 36629
2025-01-25 12:10:23.248 | INFO     | __main__:<module>:13 - Creating project with name: 36629
2025-01-25 12:10:23.250 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-25 12:10:23.272 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'f783c72b38e587ece984c3e31fff1bee40c0f5bf0f19dd8165b75fbaba582766'
2025-01-25 12:10:23.273 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9
2025-01-25 12:10:23.275 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-25 12:10:23.277 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_project_account"
2025-01-25 12:10:23,289 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-25 12:10:23,291 - INFO - ('ENOUGH_SIGNATURES_COLLECTED',

b'f541da159b13219d9020a4b0f7bd13f992091ee063599135661da57b7c9313c4'

3 - Queries the blockchain to confirm the succesful creation of the account;

In [9]:
#Query - GetAccountDetail

# - Get account info
project_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
integration_helpers.get_engine_receipts_result(project_account_hash)


2025-01-25 12:10:30.602 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-25 12:10:30,609 - WARNING - create_account_contract_address: 44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9
2025-01-25 12:10:30,633 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-25 12:10:30,636 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-25 12:10:31,847 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-25 12:10:31,857 - INFO - ('COMMITTED', 5, 0)
2025-01-25 12:10:31,859 - WARNING - Account Data: {'address': '44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9', 'hex_hash': '08dbb2e4a3aa755562d4d91fe1c586c7ccd5519d27cfe98d36d2f72ec12b4354'}
2025-01-25 12:10:31.861 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-25 12:10:31,867 - INFO - Appended data under '36629@test' in logs/account_data.json
2025-01-25 12:10:31.870 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-25 12:10:31.876 | DEBUG    | integration_he

'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00J{"account_id":"36629@test","domain_id":"test","quorum":1,"json_data":"{}"}\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [10]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-25 12:13:37,632 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-25 12:13:37.635 | INFO     | __main__:<module>:3 - b'8ab0f4ff1b75e1c1a1102b5eaa5854b0c23507747e387a15df0e03e57699a1f7'


5 - Sets details for the project account:

In [11]:
# Generate synthetic project metadata JSON (assumed to be done already)
project_metadata_json = generate_synthetic_entry_ld()
project_metadata_cid = upload_json_to_ipfs(project_metadata_json)  # Assuming this is done before calling the function

# Call the function to update or append a new entry to the JSON-LD file
updated_entry_count = update_or_append_project_metadata(
    project_id, 
    project_metadata_json, 
    project_metadata_cid
)


address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

# Setting account detail using the generated project metadata CID
hash = set_account_detail(
    address, 
    f"{project_id}@{DOMAIN}",  # Project ID as account
    "project_metadata_cid",    # The key we're setting
    project_metadata_cid       # The value (CID from IPFS)
)

2025-01-25 12:14:41.794 | INFO     | dump_to_json:update_or_append_project_metadata:216 - Updated existing project entry for project ID: 36629@test
2025-01-25 12:14:41.798 | INFO     | dump_to_json:update_or_append_project_metadata:235 - Current total entries in 'datasets/projects.json': 17
2025-01-25 12:14:41.800 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-25 12:14:41.814 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'8ab0f4ff1b75e1c1a1102b5eaa5854b0c23507747e387a15df0e03e57699a1f7'
2025-01-25 12:14:41.815 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: 2A2B7EBDCCD639978CAB491B16ECAD343A56AF42
2025-01-25 12:14:41.816 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-25 12:14:41.820 | DEBUG    | integration_helpers:tracer:35 - 	Entering "set_account_detail"
2025-01-25 12:14:41,835 - INFO - No

6 - Queries the blockchain to confirm the succesful details setting

In [12]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, project_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
user_json_ld_cid = process_raw_data(details) #TO DO - From now

2025-01-25 12:16:42.692 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-25 12:16:42,696 - WARNING - create_account_contract_address: 44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9
2025-01-25 12:16:42,724 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-25 12:16:42,727 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-25 12:16:44,379 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-25 12:16:44,390 - INFO - ('COMMITTED', 5, 0)
2025-01-25 12:16:44,392 - WARNING - Account Data: {'address': '44D5169FCDBA68A0431CC6B3CEEAFE63969B2BA9', 'hex_hash': 'fb94d21cdab3c75bfaa99773dac44e8c7bf1715b0e53d95f41c1f866f0c4b37f'}
2025-01-25 12:16:44.394 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-25 12:16:44,401 - INFO - Appended data under '36629@test' in logs/account_data.json
2025-01-25 12:16:44.404 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-25 12:16:44.405 | DEBUG    | integration_he

7 - Retrieves Project metadata (JSON-LD formatted) from IPFS

In [ ]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
project_metadata_cid = details_dict["admin@test"]["project_metadata_cid"]

logger.info(f"Project Metadata CID: {project_metadata_cid}")

project_metadata = download_json_from_ipfs(project_metadata_cid)

logger.info(f"Project Metadata: {project_metadata}")